In [1]:
import numpy as np
import timeit
import os
import ctypes
import numba as nb
import math


In [2]:
np.random.seed(0)
x = np.random.randint(0, 2147483646, 20, dtype=np.int32)
x

array([ 209652396,  398764591,  924231285, 1478610112,  441365315,
       1537364731,  192771779, 1491434855, 1819583497,  530702035,
        626610453, 1650906866, 1879422756, 1277901399, 1682652230,
        243580376, 1991416408, 1171049868, 1646868794, 2051556033])

In [3]:
all_mehods = []

In [4]:
def sumSqrtPurePython(x):
    sum = 0
    for e in x:
        sum += e ** 0.5
    return sum

all_mehods.append({
    'method': 'Pure Python',
    'func': sumSqrtPurePython,
    'times': []
})

# quick test:
sumSqrtPurePython(x)

647829.590943377

In [5]:
def sumSqrtMath(x):
    sum = 0
    for e in x:
        sum += math.sqrt(e)
    return sum

all_mehods.append({
    'method': 'Math Module',
    'func': sumSqrtMath,
    'times': []
})

# quick test:
sumSqrtMath(x)

647829.590943377

In [6]:
def sumSqrtNumpy(x):
    return np.sum(np.sqrt(x))

all_mehods.append({
    'method': 'Numpy',
    'func': sumSqrtNumpy,
    'times': []
})

# quick test:
sumSqrtNumpy(x)

647829.5909433769

In [7]:
@nb.njit(parallel=True)
def sumSqrtNumbaParallel(x):
    n = len(x)
    sum = 0
    for i in nb.prange(n):
        sum += x[i] ** 0.5
    return sum

all_mehods.append({
    'method': 'Numba parallel',
    'func': sumSqrtNumbaParallel,
    'times': []
})

# quick test:
sumSqrtNumbaParallel(x)

647829.590943377

In [8]:
@nb.njit
def sumSqrtNumba(x):
    n = len(x)
    sum = 0
    for i in range(n):
        sum += x[i] ** 0.5
    return sum

all_mehods.append({
    'method': 'Numba',
    'func': sumSqrtNumba,
    'times': []
})

# quick test:
sumSqrtNumba(x)

647829.590943377

In [13]:
# compiled functions
gcc_path = "C:\\__APP__\\mingw64\\bin\\gcc"
current_path = os.getcwd()

options = ['-O0', '-O1', '-O2', '-O3', '-Os', '-Ofast', '-Og', '-Oz']
sum_sqrt_libs = []
names = []

compilation_times = {}
for opt in options:
    t = timeit.timeit(lambda: os.system(f'{gcc_path} -shared {opt} -o sumSqrt{opt}.so sumSqrt.c'), number=1)
    if t<0.5:
        repeat = int(0.5/t)+1
        final_t = timeit.timeit(lambda: os.system(f'{gcc_path} -shared {opt} -o sumSqrt{opt}.so sumSqrt.c'), number=repeat)/repeat
    else:
        repeat = 1
        final_t = t
    compilation_times[opt] = final_t
    
    os.system(f'{gcc_path} -shared {opt} -o sumSqrt{opt}.so sumSqrt.c')
    lib = ctypes.cdll.LoadLibrary(os.path.join(current_path, f'sumSqrt{opt}.so'))
    lib.sumSqrtC.argtypes = (ctypes.POINTER(ctypes.c_int), ctypes.c_int)
    lib.sumSqrtC.restype = ctypes.c_double
    sum_sqrt_libs.append(lib)
    names.append(f'compiled{opt}')


def np_to_c(arr: np.ndarray) -> tuple[ctypes.POINTER(ctypes.c_int), ctypes.c_int]:
    return arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int)), ctypes.c_int(len(arr))

def sumSqrtC(x, lib):
    return lib.sumSqrtC(*np_to_c(x))

all_mehods.append({
    'method': 'C -O0',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[0]),
    'times': []
})
all_mehods.append({
    'method': 'C -O1',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[1]),
    'times': []
})
all_mehods.append({
    'method': 'C -O2',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[2]),
    'times': []
})
all_mehods.append({
    'method': 'C -O3',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[3]),
    'times': []
})
all_mehods.append({
    'method': 'C -Os',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[4]),
    'times': []
})
all_mehods.append({
    'method': 'C -Ofast',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[5]),
    'times': []
})
all_mehods.append({
    'method': 'C -Og',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[6]),
    'times': []
})
all_mehods.append({
    'method': 'C -Oz',
    'func': lambda x: sumSqrtC(x, lib=sum_sqrt_libs[7]),
    'times': []
})

print('Compilation times:')
for opt, t in compilation_times.items():
    print(f'{opt}: {t:.3f} s')

# quick test:
sumSqrtC(x, lib=sum_sqrt_libs[0])

Compilation times:
-O0: 0.226 s
-O1: 0.242 s
-O2: 0.270 s
-O3: 0.257 s
-Os: 0.257 s
-Ofast: 0.254 s
-Og: 0.219 s
-Oz: 0.245 s


647829.590943377

In [15]:
os.system('python setup.py build_ext --inplace')

import sumSqrtCython

all_mehods.append({
    'method': 'Cython',
    'func': sumSqrtCython.sumSqrtCython,
    'times': []
})

# quick test:
sumSqrtCython.sumSqrtCython(x)

647829.5909433769